In [70]:
!pip install moviepy opencv-python

In [80]:
import os
import pandas as pd
import cv2
import subprocess

from IPython.display import display, Video
from moviepy.editor import VideoFileClip
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

In [4]:
file_path = "../confusion_lara/whole_dataset_noproblems_feedback.json"

df = pd.read_json(file_path)

In [28]:
df.head(20)

,video,interaction,recipe,turn,utterance,start,end,speaker,text,start_i,end_i,feedback,type,feedback_value
0,2023-04-13 10-58-57,1,toffee,1,1,27.048,30.971,robot,"Hallo, ik ben Toffee Bot.Hoe heet jij?",0.0,1.0,no,greeting,None
1,2023-04-13 10-58-57,1,toffee,1,2,31.031,33.373,user,Robin.,NaN,NaN,no,greeting,None
2,2023-04-13 10-58-57,1,toffee,1,3,33.433,44.983,robot,Fijn jou te ontmoeten.Ik ben een robotkok en i...,3.0,5.0,no,interruption,None
3,2023-04-13 10-58-57,1,toffee,1,3b,45.063,47.786,user,"Oei, daar weet ik niet zo heel veel van.",NaN,NaN,no,interruption,None
4,2023-04-13 10-58-57,1,toffee,1,4,47.806,66.487,robot,"Toffee is een soort taai, hard snoepgoed, mees...",7.0,8.0,no,interruption,None
5,2023-04-13 10-58-57,1,toffee,1,4b,66.567,0.000,user,,NaN,NaN,yes,interruption,NO
6,2023-04-13 10-58-57,1,toffee,2,5,66.567,73.213,robot,Ik zie dat ik je onderbroken heb.Ik wil me hie...,9.0,10.0,no,apology_formal,None
7,2023-04-13 10-58-57,1,toffee,2,5b,73.273,74.474,user,Dat is geen probleem.,NaN,NaN,yes,apology_formal,YES
8,2023-04-13 10-58-57,1,toffee,3,6,78.263,91.511,robot,Voor dit recept heb je zelfs geen suiker nodi...,12.0,14.0,no,instruction_question,None
9,2023-04-13 10-58-57,1,toffee,3,7,91.551,96.654,user,Nee.,NaN,NaN,yes,instruction_question,YES


In [81]:
def get_number_before(number: str) -> int:
    if number.isdigit():
        return int(number) - 1
    elif number[0].isdigit():
        try:
            return int(number[:-1])
        except:
            print("Wrong format of number")

def show(video_path: str) -> None:
    """
    Display a video given a videopath
    """
    width = 700
    height = 400

    output_path = "/project_ghent/Master-Thesis/featureExtraction/out.mp4"
    
    command = [
            "ffmpeg",
            "-y",
            "-i", video_path,
            "-c:v", "libx264",
            "-crf", "23",
            "-c:a", "aac",
            "-strict", "experimental",
            output_path
        ]

    subprocess.run(command, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    
    display(Video(output_path, width=width, height=height, embed=True))

In [87]:
cropped_root = "/project_ghent/Master-Thesis/no_robot_audio/labelled/CROPPED/"
dest_root = "/project_ghent/Master-Thesis/featureExtraction/output_video_after_robot_utterance/"
VIDEO_LENGTH = 120

df_robot = df[(df["speaker"] == "robot")]
for i, row in df_robot.iterrows():
    video_endpath = str(row["video"]) + "_interaction_" + str(row["interaction"]) + "_utterance_" + str(row["utterance"]) + "_merged_cropped.mp4"
    video_path = cropped_root + "YES/" + video_endpath
    output_video_file = dest_root + "YES/" + video_endpath
    if not os.path.exists(video_path):
        video_path = cropped_root + "NO/" + video_endpath
        output_video_file = dest_root + "NO/" + video_endpath
        
    if os.path.exists(video_path):
        # show(video_path)
        
        video_capture = cv2.VideoCapture(video_path)
        width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = int(video_capture.get(cv2.CAP_PROP_FPS))
        v_len = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        output_video = cv2.VideoWriter(output_video_file, fourcc, fps, (width, height))

        start = int(row["end"] - VIDEO_LENGTH // 2)
        end = int(row["end"] + VIDEO_LENGTH // 2)
        
        amount_of_frames = 0
        if end >= v_len:
            start -= end - (v_len - 1)
            end = v_len - 1
        elif start < 0:
            end -= start
            start = 0

        count = 0
        while True:
            ret, frame = video_capture.read()
            if not ret:
                break

            if count >= start and count <= end:
                output_video.write(frame)
                amount_of_frames += 1
            elif count > end:
                break
            count += 1

        output_video.release()
        print(f"{video_path} -> {output_video_file}")
        # show(output_video_file)

/project_ghent/Master-Thesis/no_robot_audio/labelled/CROPPED/NO/2023-04-13 10-58-57_interaction_1_utterance_4_merged_cropped.mp4 -> /project_ghent/Master-Thesis/featureExtraction/output_video_after_robot_utterance/NO/2023-04-13 10-58-57_interaction_1_utterance_4_merged_cropped.mp4
/project_ghent/Master-Thesis/no_robot_audio/labelled/CROPPED/YES/2023-04-13 10-58-57_interaction_1_utterance_5_merged_cropped.mp4 -> /project_ghent/Master-Thesis/featureExtraction/output_video_after_robot_utterance/YES/2023-04-13 10-58-57_interaction_1_utterance_5_merged_cropped.mp4
/project_ghent/Master-Thesis/no_robot_audio/labelled/CROPPED/YES/2023-04-13 10-58-57_interaction_1_utterance_6_merged_cropped.mp4 -> /project_ghent/Master-Thesis/featureExtraction/output_video_after_robot_utterance/YES/2023-04-13 10-58-57_interaction_1_utterance_6_merged_cropped.mp4
/project_ghent/Master-Thesis/no_robot_audio/labelled/CROPPED/YES/2023-04-13 10-58-57_interaction_1_utterance_8_merged_cropped.mp4 -> /project_ghent/Ma

[mov,mp4,m4a,3gp,3g2,mj2 @ 0x55d451bd7640] moov atom not found


/project_ghent/Master-Thesis/no_robot_audio/labelled/CROPPED/YES/2023-04-27 12-05-59_interaction_1_utterance_15_merged_cropped.mp4 -> /project_ghent/Master-Thesis/featureExtraction/output_video_after_robot_utterance/YES/2023-04-27 12-05-59_interaction_1_utterance_15_merged_cropped.mp4
/project_ghent/Master-Thesis/no_robot_audio/labelled/CROPPED/YES/2023-04-27 12-05-59_interaction_1_utterance_17_merged_cropped.mp4 -> /project_ghent/Master-Thesis/featureExtraction/output_video_after_robot_utterance/YES/2023-04-27 12-05-59_interaction_1_utterance_17_merged_cropped.mp4
/project_ghent/Master-Thesis/no_robot_audio/labelled/CROPPED/YES/2023-04-27 12-05-59_interaction_1_utterance_18_merged_cropped.mp4 -> /project_ghent/Master-Thesis/featureExtraction/output_video_after_robot_utterance/YES/2023-04-27 12-05-59_interaction_1_utterance_18_merged_cropped.mp4
/project_ghent/Master-Thesis/no_robot_audio/labelled/CROPPED/YES/2023-04-27 12-05-59_interaction_1_utterance_20_merged_cropped.mp4 -> /project